# Práctico: Recomendación de Artistas

En este práctico trabajaremos con el conjuto de datos de [LastFM](https://grouplens.org/datasets/hetrec-2011/) para el desarrollo de un sistema de recomendación que, dado el nombre de un artista musical, devuelve una lista de artistas "similares".

Para el práctico utilizaremos el conjunto de datos de LastFM que consiguieron del [notebook de instalación](./instalacion.ipynb). Se recomienda leer el [Readme](http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-readme.txt) de los datos para saber más sobre que información contiene cada archivo.

In [1]:
import pandas as pd

In [2]:
artists = pd.read_csv(r'C:\Users\Tomas\Notebooks Python\Diplo Datos\SistemasDeRecomendacion-master\artists.dat', sep="\t")
# contiene informacion sobre los artistas

tags = pd.read_csv(r'C:\Users\Tomas\Notebooks Python\Diplo Datos\SistemasDeRecomendacion-master\tags.dat', sep="\t", encoding='latin-1')
# contiene todos los tags posibles, es decir las categorias a las que pueden ser asignados los artistas

user_artist_plays = pd.read_csv(r'C:\Users\Tomas\Notebooks Python\Diplo Datos\SistemasDeRecomendacion-master\user_artists.dat', sep="\t")
# contiene la cantidad de veces que determinado usuario escucho determinado artista

user_artist_tags = pd.read_csv(r'C:\Users\Tomas\Notebooks Python\Diplo Datos\SistemasDeRecomendacion-master\user_taggedartists.dat', sep="\t")
# contiene el tag que cada usuario le asigno a determinado artista y el año en que lo hizo 

user_relation = pd.read_csv(r'C:\Users\Tomas\Notebooks Python\Diplo Datos\SistemasDeRecomendacion-master\user_friends.dat', sep="\t")
# contiene las relaciones entre los usuarios de los dataset


# al rating lo tenemos que calcular nosotros, puedo usar esto solo o puedo usar otras cosas. Osea puedo calcular el rating 
# usando unicamente como feature la cantidad de veces que el usuario escucho al artista o puedo incorporar otras cosas tambien
# eso depende de como lo quiera hacer yo

In [3]:
user_relation.head()

,userID,friendID
0,2,275
1,2,428
2,2,515
3,2,761
4,2,831


## Actividades

El [Ejercicio 1](#Ejercicio-1---Análisis-Exploratorio-de-Datos) deberá ser realizado por quienes estén realizando cualquier parte de la materia.

El [Ejercicio 2](#Ejercicio-2---Sistema-de-Recomendación) variará de acuerdo a que parte de la materia estén realizando (quienes estén realizando la materia completa, en realidad pueden realizar ambas opciones si así lo desean).

De acuerdo a la parte de la materia que hagan, deberán realizar una de las siguientes actividades (pueden realizar ambas si así lo desean):

La idea del práctico es hacer un análisis muy sencillo del conjunto de datos y desarrollar dos sistemas de recomendación: 
1. El primero, más sencillo, utilizando [Surpr!se](http://surpriselib.com/), y a partir de los datos de LastFM, en especial del archivo `./data/lastfm/user_artists.dat`, generar un sistema de recomendación basado en filtros colaborativos.
2. En el segundo, deberán utilizar todos los datos ofrecidos en el dataset de LastFM para generar un sistema de filtrado colaborativo más complejo, deberá utilizar las técnicas aprendidas 

basado en filtrado colaborativo (usando Surpr!se), a partir de los datos existentes.

## Ejercicio 1 - Análisis Exploratorio de Datos

En esta primera parte deberán hacer un análisis exploratorio de los datos, aprovechando toda la información brindada por el conjunto. A partir de eso podrán tener mayor idea de qué tipo de datos estarán enfrentando (describe o hist).

Algunas preguntas para responder:
- ¿Cuáles son los artistas que fueron más escuchados?
- ¿Cómo es la distribución de cantidad de listens por user?
- ¿Es posible ver el género más escuchado?

In [4]:
# Para saber cual es el artista más escuchado

plays = user_artist_plays.groupby(['artistID'])['weight'].agg('sum').reset_index()
plays = plays.merge(artists, left_on = 'artistID', right_on = 'id')
plays = plays.iloc[:,[0,1,3]]
plays = plays.sort_values('weight', ascending = False)
plays2 = user_artist_plays.groupby('artistID')['userID'].count().reset_index()
plays = plays.merge(plays2, left_on = 'artistID', right_on = 'artistID')
plays = plays.rename(columns={ 'artistID' : 'artistID', 'weight':'total_listens', 'name':'artist_name', 'userID' : 'n_of_listeners'})
plays.head(10)



,artistID,total_listens,artist_name,n_of_listeners
0,289,2393140,Britney Spears,522
1,72,1301308,Depeche Mode,282
2,89,1291387,Lady Gaga,611
3,292,1058405,Christina Aguilera,407
4,498,963449,Paramore,399
5,67,921198,Madonna,429
6,288,905423,Rihanna,484
7,701,688529,Shakira,319
8,227,662116,The Beatles,480
9,300,532545,Katy Perry,473


La artista más escuchada fue Britney Spears, con casi 2 millones y medio de reproducciones. En segundo lugar, pero a más de un millón de reproducciones de Britney, se encontró Depeche Mode seguido de cerca por Lady Gaga. El top 6 lo completan Christina Aguilera, Paramore y Madonna.
Si bien Gaga es la tercer artista más escuchada según cantidad de reproducciones, es la artista que fue más escuchada en cuanto a cantidad de personas: 611 personas distintas escucharon sus canciones. La segunda artista más escuchada por diferentes personas fue Britney Spears, que fue reproducida por 522 usuarios. 

In [5]:
from plotly.offline import init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

data = plays.loc[plays['n_of_listeners'] > 10, :]

trace = go.Histogram(x = data.n_of_listeners.values,
                     name = 'Listeners',
                     xbins = dict(start = 0,
                                  end = 650,
                                  size = 5))
# Create layout
layout = go.Layout(title = 'Distribution of number of listeners per artist',
                   xaxis = dict(title = 'Number of Listeners per artist'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

En este histograma se puede ver que la gran mayoría de los artistas son escuchados por un pequeño número de personas. De los más de 17 mil artistas sólo alrededor de 125 fueron escuchados por más de 100 personas, y la gran mayoría fueron escuchados por menos de 15 personas. 

In [6]:
# Para conocer la distribucion de cantidad de listens por users

users = user_artist_plays.groupby("userID").sum()['weight'].reset_index().sort_values('weight', ascending = False)

users2 = user_artist_plays.groupby("userID").mean()['weight'].reset_index()

users3 = user_artist_plays.groupby("userID")['artistID'].nunique().reset_index()

users = users.merge(users3, left_on = 'userID', right_on = 'userID')
users = users.merge(users2, left_on = 'userID', right_on = 'userID')

users = users.rename(columns={ 'userID' : 'userID', 'weight_x':'total_listens', 'weight_y':'mean_listens_per_artist',
                              'artistID' : 'artist_per_user'})
users


,userID,total_listens,artist_per_user,mean_listens_per_artist
0,757,480039,50,9600.78
1,2000,468409,50,9368.18
2,1418,416349,50,8326.98
3,1642,388251,50,7765.02
4,1094,379125,50,7582.50
...,...,...,...,...
1887,1334,5,4,1.25
1888,1893,4,4,1.00
1889,2085,4,1,4.00
1890,188,4,2,2.00


In [7]:
trace = go.Histogram(x = users.artist_per_user.values,
                     name = 'Users',
                     xbins = dict(start = 0,
                                  end = 50,
                                  size = 1))
# Create layout
layout = go.Layout(title = 'Distribution of number of artist per user',
                   xaxis = dict(title = 'Number of artist per user'),
                   yaxis = dict(title = 'Count'),
                   bargap = 0.2)

# Create plot
fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

La gran mayoria de los usuarios (más de 1500) escucharon a 50 artistas distintas. De hecho la distribución es tan desproporcional que para que el histograma sea interpretable hay que dejar afuera a quellos usuarios que escucharon a 50 artistas distintos. Lo curioso es que ninguno de todos ellos escuchó más de 50, como si ese hubiese sido la cantidad máxima de aritstas diferentes que se podía escuchar. 

In [8]:
# para obtener el genero más escuchado (medido en cantidad de usuarios distintos que esucharon ese género, no en cantidad de
# de reproducciones)

genero = user_artist_tags.iloc[:, [0,1,2]]
genero = genero.groupby("tagID")["userID"].count().reset_index().sort_values("userID", ascending = False)
genero = genero.merge(tags, left_on = "tagID", right_on = "tagID")
genero = genero.rename(columns = {'tagID':'tagID', 'userID': 'n_of_listeners', 'tagValue': 'tagValue' })
genero2 = user_artist_tags.merge(user_artist_plays, left_on = ["userID", "artistID"], right_on = ["userID", "artistID"])
genero2 = genero2.iloc[:,[0,1,2,6]]
genero2 = genero2.groupby("tagID").sum()['weight'].reset_index().sort_values("weight", ascending = False)
genero = genero.merge(genero2, left_on = "tagID", right_on = "tagID")
genero = genero.rename(columns = {'tagID':'tagID', 'userID': 'n_of_listeners', 'tagValue': 'tagValue', 'weight': 'reproducciones'})
genero

,tagID,n_of_listeners,tagValue,reproducciones
0,73,7503,rock,5081342
1,24,5418,pop,6208564
2,79,5251,alternative,3057909
3,18,4672,electronic,3422492
4,81,4458,indie,1466077
...,...,...,...,...
7922,7452,1,finding your way,9
7923,7451,1,divorce,9
7924,7450,1,soulmate,9
7925,7449,1,crush,9


El genero más escuchado según la cantidad de usuarios fue el rock con 7503 usuarios, seguido por el pop con 5418. Sin embargo, el género más escuchado según cantidad de reproducciones fue primero el pop y segundo el rock. 

## Ejercicio 2 - Sistema de Recomendación

### Ejercicio 2a - Filtrados Colaborativos

Esta parte del ejercicio es obligatoria para quienes quieran aprobar la parte introductoria de la materia (i.e. los contenidos que se ven en las dos primeras clases), quienes estén realizando la materia completa pueden optar por saltearse este ejercicio (aunque es recomendable pensarlo) y pasar directamente al [Ejercicio 2b](#Ejercicio-2b---Sistemas-de-Recomendación-Avanzados). Deberán realizar un sistema de filtrados colaborativos basado en [Surpr!se](http://surpriselib.com/), a partir de los datos que proporciona `LastFM`, en especial el archivo `user_artists.dat`. Tener en cuenta los siguientes pasos:

1. **Desarrollo de la matriz de Usuario-Contenido:** A partir del conjunto de datos deberán generar una matriz de usuario-contenido. Tener en cuenta que los ratings son implícitos, puesto que se dan a partir de la cantidad de veces que un usuario escuchó a determinado artista.
2. **Entrenamiento del algoritmo de recomendación**: Utilizando las herramientas brindadas por [Surpr!se](http://surpriselib.com/), entrenen varios modelos (al menos 3) de sistemas de recomendación basados en filtrado colaborativo a partir de su matriz de usuario-contenido. Recuerden tener en cuenta lo aprendido en la diplomatura a la hora de evaluar y validar el modelo. Si necesitan inspiración, les recomendamos revisar [este notebook con información de como entrenar un sistema de recomendación con Surpr!se](https://github.com/susanli2016/Machine-Learning-with-Python/blob/master/Building%20Recommender%20System%20with%20Surprise.ipynb).
3. **Sistema de recomendación**: A partir del mejor modelo de recomendación que haya surgido del caso anterior, y utilizando los datos del archivo `artist.dat`, armar un sistema de recomendación sencillo que, dado un nombre de un artista, devuelva el top 10 de artistas más similares. La idea es que el sistema tome el nombre de un artista y devuelva el nombre de otros artistas (no simplemente tomar y devolver IDs). Se recomienda [revisar este notebook para inspiración (ver el paso número 5)](https://github.com/topspinj/pydata-workshop/blob/master/tutorial.ipynb).

In [9]:
from surprise import Dataset, Reader
from surprise.accuracy import rmse # osea para medir la accuracy voy a usar el Error Cuadratico Medio
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy

# Importo algoritmos que voy a usar 

from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering

Para reducir la dimensionalidad del dataset, filtramos a los artistas que fueron escuchados por menos de 5 personas al igual que a las personas que escucharon a menos de 5 artistas

In [10]:
filter_artists = plays.loc[plays['n_of_listeners'] > 5, :]

filter_users = users.loc[users['artist_per_user'] > 5, : ]

df_new = user_artist_plays[(user_artist_plays['artistID'].isin(filter_artists['artistID'])) & (user_artist_plays['userID'].isin(filter_users['userID']))]
print('The original data frame shape:\t{}'.format(user_artist_plays.shape))
print('The new data frame shape:\t{}'.format(df_new.shape))

The original data frame shape:	(92834, 3)
The new data frame shape:	(69354, 3)


In [11]:
# Normalizacion de la cantidad de escuchas (divido la cantidad de escuchas de cada persona para cada artista por la cantidad 
# maxima de esuchas de esa persona para un artista cualquiera). Esto sirve para bajar el valor absoluto del ECM del algoritmo
# que decida aplicar

data = df_new.groupby("userID").max()['weight'].reset_index()
data = data.rename(columns={'userID':'userID', 'weight':'max_listens'})
data = df_new.merge(data, left_on = "userID", right_on = "userID")
data['value'] = data['weight']/data['max_listens']
data = data.iloc[:,[0,1,4]]
data

,userID,artistID,value
0,2,51,1.000000
1,2,52,0.842037
2,2,53,0.817619
3,2,54,0.741915
4,2,55,0.647050
...,...,...,...
69349,2100,3806,0.192004
69350,2100,4271,1.000000
69351,2100,4611,0.177690
69352,2100,6258,0.198914


In [12]:
# antes de hacer el reader normalizar los valores. COMO? dividiendo la cantidad de escuchas de cada personas por el valor 
# maximo de escuchas de esa misma persona. 

reader = Reader(rating_scale=(data.value.min(), data.value.max()))

df = Dataset.load_from_df(data[['userID', 'artistID', 'value']], reader)

## Entrenamiento de los modelos

A continuación se entrenarán diferentes algoritmos sobre el conjunto de datos que contiene a los usuarios, los artistas y la cantidad de veces que cada usuario escuchó cada artista. La métrica que se utilizará para evaluar qué tan bueno es cada modelo será el Error Cuadrático Medio (RMSE).

In [13]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, df, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


In [14]:
surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

surprise_results

,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.193295,34.827225,1.669972
KNNWithMeans,0.194177,0.720886,4.135455
KNNBaseline,0.197875,0.729215,4.982267
KNNWithZScore,0.198655,0.906773,4.555530
SVD,0.205704,4.399624,0.498026
KNNBasic,0.220180,0.558654,3.907262
CoClustering,0.283155,2.476474,0.215867


Si bien el modelo con el error cuadrático medio más bajo es el SVD, optamos por aplicar el KNNWithMeans porque está más dentro de la línea del filtrado colaborativo. 

In [15]:
data_train, data_test = train_test_split(df, test_size=0.2)

# esto es mas o menos lo mismo de siempre, calculo mi modelo con KNN con K=5. Lo entreno sobre el conjunto de train y despues 
# lo pongo a prueba sobre el conjunto de test. Finalemnte calculo el error cuadratico medio

model = KNNWithMeans(k=5).fit(data_train)
predictions = model.test(data_test)
print("RMSE on test: {:.4f}".format(rmse(predictions, verbose=False)))

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE on test: 0.2066


In [16]:
model.predict(12, 27)

# el modelo estima que el usuario 12 escucho al artista 27 solo un 30% de las veces que escucho a su artista mas escuchado

Prediction(uid=12, iid=27, r_ui=None, est=0.06157970874899967, details={'actual_k': 2, 'was_impossible': False})

In [17]:
model = KNNWithMeans(k=5, verbose=False)
cross_validated_metrics = cross_validate(model, df, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.2072  0.2079  0.2027  0.2089  0.2040  0.2061  0.0024  
MAE (testset)     0.1287  0.1286  0.1271  0.1295  0.1278  0.1283  0.0008  
Fit time          0.83    0.81    0.81    0.82    1.22    0.90    0.16    
Test time         2.36    2.19    1.84    1.85    2.26    2.10    0.22    


### Sistema de Recomendación

Para obtener una lista de recomendación de artistas basados en el modelo anterior, simplemente completar en artist_name con el nombre del artista del cual se quieren conocer otros artistas parecidos.

In [20]:
artists_names = dict(zip(artists['id'], artists['name']))

artist_name = "Britney Spears"

artist_id = artists.loc[artists['name'] == artist_name, 'id'].iloc[0]
n_neighbors = 10

similar_artists = model.get_neighbors(artist_id, n_neighbors)
#artist_name = zip(artists['id'] == artist_id, artists['name'])

print(f"Because you listened {artist_name}:")
for i in similar_artists:
    print(artists_names[i])

Because you listened Britney Spears:
Scissor Sisters
Agonoize
Digitalism
Sarah Vaughan
Tequila Baby
Joanna Newsom
Shawn Lee's Ping Pong Orchestra
Georg Friedrich Händel
Ne-Yo
Jamelia


### Ejercicio 2b - Sistemas de Recomendación Avanzados

Este ejercicio lo deberán completar quienes hayan realizado ambas partes de la materia pues requiere de los conocimientos adquiridos en las segundas dos clases. En este caso, utilizarán no sólo la información de la matriz de usuario-contenido, sino que deberán hacer uso de otra información para poder lidiar con el problema del "Cold Start", i.e. que es lo que ocurre cuando tengo nuevos usuarios o artistas. Tener en cuenta los siguientes pasos:
1. **Evaluación sobre cold start**: Para evaluar como funciona el sistema con el problema del "cold start", deberán tomar ciertos artistas y "dejarlos fuera", i.e. remover cualquier participación del artista en el sistema. Esos serán los artistas que se evaluarán como "cold start".
2. **Vectores de contenido**: Deberán generar vectores de contenido para los artistas, basados en los tags que los usuarios les dan (ver los archivos `user_taggedartists.dat` y `tags.dat`).
3. **Sistema de recomendación**: Deberán crear un sistema de recomendación que tomará como parámetros la información del artista (i.e. nombre y tags). Con dicha información, deberán disponer de un sistema de recomendación híbrido (utilizando cualquiera de las técnicas vistas en clase) que devuelva artistas similares. El sistema de recomendación deberá utilizar toda la información proporcionada para dar una mejor respuesta.
4. **Evaluación del sistema**: Deberán evaluar "a mano" el sistema sobre artistas que conozca y artistas que no conozca (i.e. que fueron dejados afuera), y hacer un análisis de lo que el sistema está devolviendo. Osea por ejemplo si el artista que saque completamente es Madona, y yo se que Madona es parecido a Michael Jackson, cuando le meta a Madona al conjunto de datos y le pida que me devuelva un artista similar tengo que corroborar que me devuelva Michael Jackson

In [ ]:
# Para hacer lo de cold start directamente saco artistas del dataset, para no tener ningun tipo de informacion sobre ellos.
# osea en el conjunto de entrenamiento tengo que sacar todos algunos artistas (10) y sacar TODA la informacion que aparezca
# de ese artista en el conjunto de entrada. 
# Con los artistas que me quedan genero los vectores de contenidos, y con eso voy a gnerar un sistema hibrido (no solo basado 
# filtros colaborativos sino tambien en filtrado por contenido)
# GENERO EL SISTEMA HIBRIDO CON EL DATASET SIN ESOS ARTISTAS
# despues le doy de comer al sistema de recomendacion un artista que originalmente no estaba en el dataset CON SU GENERO (tags)
# y el sistema de recomendacion me va a devolver artistas parecido a el 
# el conjunto de test seria armar un dataset de artistas que no aparecian en el conjunto de entrenamiento en absoluto combinado
# con algunos artistas que si estaban en el dataset de entrenamiento


# la matriz de contenido puede ser una de ceros y unos (dandole un uno a los artistas que si pertenecen a ciertos tags) o puedo
# darle un determinado peso

# la mejor forma de armar la matriz de contenido es DictVectorizer de scikit surprise

# o sino con un pivot_table que el indice sean los artistos y las columnas son los tags (antes de eso talvez sea conveniente 
# eliminar todos los tags que solo tienen un artista puesto en ellos, para no tener tantas columnas despues)